In [2]:
%pip install indic-nlp-library

     -------------------------------------- 40.3/40.3 kB 112.9 kB/s eta 0:00:00
     ---------------------------------------- 7.7/7.7 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 1.4 MB/s eta 0:00:00
     ------------------------------------ 587.4/587.4 kB 947.7 kB/s eta 0:00:00
     -------------------------------------- 121.1/121.1 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 119.3/119.3 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 82.5/82.5 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 98.7/98.7 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 88.7/88.7 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 92.1/92.1 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 93.0/93.0 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 9.6/9.6 MB 685.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to 


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install python-Levenshtein

     -------------------------------------- 98.5/98.5 kB 235.4 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 780.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import re
from difflib import get_close_matches
from indicnlp.tokenize.sentence_tokenize import sentence_split
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator
from Levenshtein import distance
from concurrent.futures import ThreadPoolExecutor

replacements = {
    'அ': 'a', 'ஆ': 'aa', 'இ': 'i', 'ஈ': 'ii', 'உ': 'u', 'ஊ': 'uu',
    'எ': 'e', 'ஏ': 'ee', 'ஐ': 'ai', 'ஒ': 'o', 'ஓ': 'oo', 'ஔ': 'au',
    'க': 'k', 'ச': 's', 'ட': 't', 'த': 'th', 'ப': 'p', 'ற': 'r',
    'ன': 'n', 'ந': 'n', 'ம': 'm', 'ய': 'y', 'ர': 'r', 'ல': 'l',
    'வ': 'v', 'ழ': 'zh', 'ள': 'l', 'ஜ': 'j', 'ஷ': 'sh', 'ஸ': 's', 'ஹ': 'h'
}

def phonetic_encode(word):
    encoded_word = ""
    for char in word:
        encoded_word += replacements.get(char, char)
    return encoded_word

def segment_syllables(word):
    return list(word) 

def spell_checker(input_word, tamil_dictionary):
    input_encoded = phonetic_encode(input_word)
    dictionary_encoded = {word: phonetic_encode(word) for word in tamil_dictionary}

    with ThreadPoolExecutor() as executor:
        closest_matches = executor.map(
            lambda word: (word, distance(input_encoded, dictionary_encoded[word])),
            tamil_dictionary
        )
    
    sorted_matches = sorted(closest_matches, key=lambda x: x[1])[:3]
    return [match[0] for match in sorted_matches]

def check_grammar(sentence):
    GRAMMAR_RULES = {
        "simple_sentence": (re.compile(r"^(?:\u0BA8\u0BBE\u0BA9\u0BCD|\u0BA8\u0BC0|\u0BAA\u0B9F\u0BAE\u0BCD|\u0BAE\u0BB4\u0BC8|\u0BAE\u0BB2\u0BB0\u0BCD|\u0BAF\u0BBE\u0BB0\u0BCD|\u0BB0\u0BAE\u0BA9\u0BCD)\s+\S+\s+(?:\u0B9A\u0BC8\u0BAF\u0BCD\u0B95\u0BBF\u0BB1\u0BC7\u0BA9\u0BCD|\u0B9A\u0BC8\u0BAF\u0BCD\u0B95\u0BBF\u0BB1\u0BCB\u0BA9\u0BCD)\.$"), "This is a simple sentence."),
    }

    for rule_name, (rule, message) in GRAMMAR_RULES.items():
        if rule.match(sentence):
            return f"Sentence is grammatically correct. Rule matched: {rule_name} - {message}"

    return "Sentence does not match any known grammar rule."

def tokenize_sentences(paragraph):
    return sentence_split(paragraph, lang='ta')

def integrated_checker(sentence, tamil_dictionary):
    sentence = UnicodeIndicTransliterator.transliterate(sentence, "auto", "ta")
    words = sentence.split()
    all_suggestions = {}
    corrected_sentence = []

    for word in words:
        suggestions = spell_checker(word, tamil_dictionary)
        if suggestions and word not in tamil_dictionary:
            all_suggestions[word] = suggestions
            corrected_sentence.append(suggestions[0])
        else:
            corrected_sentence.append(word)

    corrected_sentence = " ".join(corrected_sentence)

    if not (corrected_sentence.endswith(".") or corrected_sentence.endswith("।")):
        corrected_sentence += "."

    grammar_result = check_grammar(corrected_sentence)

    return all_suggestions, grammar_result, corrected_sentence

def load_tamil_dictionary(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        tamil_dictionary = [line.strip() for line in file]
    return tamil_dictionary

if __name__ == "__main__":
    tamil_dictionary = load_tamil_dictionary("C:/Users/Admin/Desktop/dictionary.txt")

    sentences = [
        "நான் படம் படிக்கிறே",
        "அனைவருக்கும் நல்வாழ்துகள். தமிழின் மிக்க பழமயான மொழி வரலாற்றையும் பெருமையும் கொண்டது. இது உலகன் மிக அழகிய மற்றும் செம்மொழிகளில ஒன்றாகும். தமிழில் சொற்களை சரயாக எழுதுவதும், சரியான இடத்தில முறையாக பயன்படுத்துதும் மிகவும் முக்கியமானவை"
    ]

    for sentence in sentences:
        segmented_sentences = tokenize_sentences(sentence)
        for segmented_sentence in segmented_sentences:
            print(f"Checking sentence: {segmented_sentence}")
            spell_suggestions, grammar_check, corrected_sentence = integrated_checker(segmented_sentence, tamil_dictionary)

            print("\nSpell Check Suggestions:")
            for word, suggestions in spell_suggestions.items():
                print(f"  {word}: {', '.join(suggestions)}")

            print("\nGrammar Check:")
            print(grammar_check)

            print("\nCorrected Sentence:")
            print(corrected_sentence)

Checking sentence: நான் படம் படிக்கிறே

Spell Check Suggestions:
  படம்: பாடம், பாடம், பணம்
  படிக்கிறே: படிக்கிறேன், படிக்க, அறிக்கை

Grammar Check:
Sentence does not match any known grammar rule.

Corrected Sentence:
நான் பாடம் படிக்கிறேன்.
Checking sentence: அனைவருக்கும் நல்வாழ்துகள்.

Spell Check Suggestions:
  நல்வாழ்துகள்.: நல்வாழ்த்துகள், காத்து, வாழ்கை

Grammar Check:
Sentence does not match any known grammar rule.

Corrected Sentence:
அனைவருக்கும் நல்வாழ்த்துகள்.
Checking sentence: தமிழின் மிக்க பழமயான மொழி வரலாற்றையும் பெருமையும் கொண்டது.

Spell Check Suggestions:
  பழமயான: பழமையான, பிரதான, இனிமையான
  கொண்டது.: கொண்டது, காத்து, பொது

Grammar Check:
Sentence does not match any known grammar rule.

Corrected Sentence:
தமிழின் மிக்க பழமையான மொழி வரலாற்றையும் பெருமையும் கொண்டது.
Checking sentence: இது உலகன் மிக அழகிய மற்றும் செம்மொழிகளில ஒன்றாகும்.

Spell Check Suggestions:
  உலகன்: உலகம், உலகம், உலகம்
  செம்மொழிகளில: செம்மொழிகளில், செய்திகள், செய்திகள்
  ஒன்றாகும்.: ஒன்றாகும், ந